# Infer Communities via SBM

### Infer best k

In [19]:
Ks = [2,3,4,5,6,7,8,9,10]
standard_objective_function = pysbm.TraditionalUnnormalizedLogLikelyhood(is_directed=False)
colors = {0:'b',1:'r',2:'g',3:'orange',4:'black',5:'lime', 6: 'tab:pink', 7: 'tab:gray', 8: 'tab:olive'}

In [ ]:
def evaluate_community_sizes(graph, Ks, iterations=5):
    best_objectives = []
    best_partitions = []
    for i, K in enumerate(Ks): 
        best_objective = -np.inf
        best_partition = None
        print(f'Current K value: {K}')
        for r in range(iterations):
            standard_partition = pysbm.NxPartition(graph=graph,number_of_blocks=K)
            standard_inference = pysbm.KarrerInference(graph, standard_objective_function, standard_partition)
            standard_inference.infer_stochastic_block_model()
            L=standard_objective_function.calculate(standard_partition)
            if L>best_objective:
                best_objective=L
                best_partition = standard_partition
            print(f'\tIteration {r} - Best objective value: {best_objective}, Current objective value: {L}')
        best_partitions.append(best_partition)
        best_objectives.append(best_objective)
        
    return best_partitions, best_objectives
        
print("Positive correlation: \n")
best_partitions_pos, best_objectives_pos = evaluate_community_sizes(graph_pos, Ks)

print("Negative correlation: \n")
best_partitions_neg, best_objectives_neg = evaluate_community_sizes(graph_neg, Ks)

Positive correlation: 

Current K value: 2
	Iteration 0 - Best objective value: -230214.85855630055, Current objective value: -230214.85855630055
	Iteration 1 - Best objective value: -230214.85855630055, Current objective value: -230214.85855630055
	Iteration 2 - Best objective value: -230214.85855630055, Current objective value: -230214.85855630055
	Iteration 3 - Best objective value: -230214.85855630055, Current objective value: -230214.85855630055
	Iteration 4 - Best objective value: -230214.85855630055, Current objective value: -242860.2955436973
Current K value: 3
	Iteration 0 - Best objective value: -202650.11055318516, Current objective value: -202650.11055318516
	Iteration 1 - Best objective value: -202650.11055318516, Current objective value: -202707.23994722898
	Iteration 2 - Best objective value: -202078.96627765676, Current objective value: -202078.96627765676
	Iteration 3 - Best objective value: -202078.96627765676, Current objective value: -202707.23994722895
	Iteration 4

## Model Selection 

In [ ]:
def find_best_k(best_objectives, Ks, reg=1000):
    # maximum likelihood approach
    ind_ml = np.argmax(best_objectives)
    k_ml = Ks[ind_ml]
    # partition_ml = best_partitions[ind_ml]

    # bic approach 
    params = np.array(Ks)**2+n
    bic_scores = np.array(best_objectives) - reg * params
    ind_bic = np.argmax(bic_scores)
    k_bic = Ks[ind_bic]
    print("best k BIC", k_bic)
    # partition_bic = best_partitions[ind_bic]
    
    print("Maximum Likelihood Scores:", best_objectives)
    print("BIC Scores:", bic_scores)
    
    return k_ml, k_bic

# k_ml_pos, k_bic_pos = find_best_k(best_objectives_pos, Ks)
# k_ml_neg, k_bic_neg = find_best_k(best_objectives_neg, Ks)

k_bic_pos = 5
k_bic_neg = 4

partition_pos = pysbm.NxPartition(graph=graph_pos,number_of_blocks=k_bic_pos)
partition_neg = pysbm.NxPartition(graph=graph_neg,number_of_blocks=k_bic_neg)